# Perceptron with numpy

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os

%matplotlib inline

: 

### Create toy dataset

In [ ]:
data = pd.read_csv('../../data/perceptron_toydata.txt',
    sep = '\t',
    header = None)

X, y = data.iloc[:, :2], data.iloc[:, 2]
y = y.astype(np.int)

# -- Shuffle randomly rows
shuffle_idx = np.arange(y.shape[0])
shuffle_rng = np.random.RandomState(42)
shuffle_rng.shuffle(shuffle_idx)
X, y = X.iloc[shuffle_idx], y.iloc[shuffle_idx]

# -- Split train (70)/ test (30)
X_train, X_test = X.iloc[shuffle_idx[:70]], X.iloc[shuffle_idx[70:]]
y_train, y_test = y.iloc[shuffle_idx[:70]], y.iloc[shuffle_idx[70:]]

: 

### Normalize train

In [ ]:
# -- Normalize train (mean zero, unit variance)
mu, sigma = X_train.mean(axis = 0), X_train.std(axis = 0)
X_train = (X_train - mu) / sigma
X_test = (X_test - mu) / sigma

: 

Check data has been normalized correctly, mean should be 0 for each feature.

In [ ]:
X_train.mean(axis = 0)

: 

### Plot train data

In [ ]:
x1 = -0.1
x2 = 0.5
b = 0
2.3823 * x1 + 0.8073 * x2 + b

: 

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (9, 4))

sns.scatterplot(data = X_train,
                x = 0,
                y = 1, 
                hue = y_train,
                ax = ax[0]).set(title ='Train dataset')

sns.scatterplot(data = X_test,
                x = 0, 
                y = 1, 
                hue = y_test,
                ax = ax[1]).set(title='Test dataset')
fig.show()

: 

### Define perceptron class

In [ ]:
class Perceptron():
    
    def __init__(self, num_features):
        self.num_features = num_features
        self.weights = np.zeros((num_features, 1), dtype=float)
        self.bias = np.zeros(1, dtype=float)
        self.weights_history =  np.zeros((1, num_features + 1), dtype=float)

    def forward(self, x):
        # Produce the prediction
        linear = np.dot(x, self.weights) + self.bias # comp. net input
        predictions = np.where(linear > 0., 1, 0)
        return predictions
        
    def backward(self, x, y):  
        # Compute the errors to adjust weights
        predictions = self.forward(x)
        errors = y - predictions
        return errors
        
    def train(self, x, y, epochs):
        for e in range(epochs):
            
            for i in range(y.shape[0]):
                errors = self.backward(x[i].reshape(1, self.num_features), y[i]).reshape(-1)
                self.weights += (errors * x[i]).reshape(self.num_features, 1)
                self.bias += errors
                # -- Keep track of weights change
                self.weights_history = np.vstack([self.weights_history, np.append(ppn.weights.reshape(-1), ppn.bias)])
                
    def evaluate(self, x, y):
        predictions = self.forward(x).reshape(-1)
        accuracy = np.sum(predictions == y) / y.shape[0]
        return accuracy

: 

### Train perceptron

In [ ]:
ppn = Perceptron(num_features = 2)

ppn.train(x = X_train.to_numpy(),
          y = y_train.to_numpy(),
          epochs = 100)

ppn.weights

: 

In [ ]:
w1 = ppn.weights[0]
w2 = ppn.weights[1]
b = ppn.bias[0]

x1 = -1
y1 = ((-(w1 * x1) - b) / w2)[0]


x2 = 1
y2 = ((-(w1 * x2) - b) / w2)[0]

: 

### Plot decision boundary

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

p1 = sns.scatterplot(data = X_train,
                     x = 0,
                     y = 1, 
                     hue = y_train,
                     ax = ax)

plt.plot([x1, x2], [y1, y2],linewidth = 2)

: 